In [1]:
import numpy as np
import os
import subprocess

%run ../potencia-deflacion/met-potencia.ipynb # me importo eigen y power_iteration
%run ../datos/datos.ipynb # me importo dataset

Código de PCA para que me genere los txt de las particiones (es el mismo algortimo que en carpeta pca)

In [2]:
def centralizar(x):
    return x - np.mean(x, axis=0)

In [3]:
def pca(A, i):
    # Compile the C++ code if needed
    subprocess.run([
        'g++', '-o', f'../val-cruzada/datos/met_potencia', '../potencia-deflacion/met_potencia.cpp',
        '-std=c++11', '-I', 'Eigen', '-O3'
    ])

    X = centralizar(A)

    # Covariance matrix
    cov_matriz = (1 / (len(X) - 1)) * np.dot(X.T, X)

    # Save covariance matrix to a file
    cov_matrix_file = f'../val-cruzada/particiones/matriz_cov_entrada{i}.txt'
    if os.path.exists(cov_matrix_file):
        os.remove(cov_matrix_file)

    with open(cov_matrix_file, 'a') as f:
        f.write(f"{cov_matriz.shape[0]} {cov_matriz.shape[1]}\n")  # Size of the matrix
        np.savetxt(f, cov_matriz, newline="\n")

    # Run the C++ program to compute eigenvalues and eigenvectors
    result = subprocess.run([
        "../val-cruzada/datos/met_potencia", cov_matrix_file
    ], stdout=subprocess.PIPE)

    # Load the results from the output file
    output_file = f'../val-cruzada/particiones/matriz_cov_salida{i}.txt'
    if os.path.exists(output_file):
        os.remove(output_file)

    with open(output_file, "a") as output:
        output.write(result.stdout.decode('utf-8'))

    arr = np.loadtxt(output_file)

    # Extract eigenvalues and eigenvectors
    eigenvalues = arr[1]  # Eigenvalues
    eigenvectors = np.delete(arr, [0, 1], axis=0)  # Eigenvectors as columns

    # Sort eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]
    autovalores_ord = eigenvalues[sorted_indices]
    autovectores_ord = eigenvectors[:, sorted_indices]

    # Save sorted eigenvectors
    eigenvector_file = f'../val-cruzada/particiones/autovectores_cov{i}.txt'
    if os.path.exists(eigenvector_file):
        os.remove(eigenvector_file)

    with open(eigenvector_file, 'a') as f:
        np.savetxt(f, autovectores_ord, newline="\n")

    # Save sorted eigenvalues
    eigenvalue_file = f'../val-cruzada/particiones/autovalores_cov{i}.txt'
    if os.path.exists(eigenvalue_file):
        os.remove(eigenvalue_file)

    with open(eigenvalue_file, 'a') as f:
        np.savetxt(f, autovalores_ord)

    return autovalores_ord, autovectores_ord